# Autoencoder LSTM Model with FAISS Evaluation

In [1]:
!pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import re
import string
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import tensorflow
import sklearn
from tqdm import tqdm
from tensorflow.keras.models import load_model
from keras_preprocessing.sequence import pad_sequences
from keras.models import Model

In [3]:
from keras.preprocessing.text import Tokenizer

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_punctuation_and_lower(text):
    text = text.lower()
    return text.translate(str.maketrans(' ', ' ', string.punctuation))

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    return [word for word in word_tokens if word not in stop_words]

def remove_usernames(text: str) -> str:
    """
    Remove usernames starting with '@' from the given text.

    Args:
        text (str): The input text containing usernames.

    Returns:
        str: The text with usernames removed.
    """
    cleaned_text = re.sub(r"@[A-Za-z0-9_]+", "", text)
    return cleaned_text

def clean_text(text):
    text = remove_html_tags(text)
    text = remove_usernames(text)
    text = remove_urls(text)
    text = remove_punctuation_and_lower(text)
    text_without_newlines = text.replace('\n', ' ')
    words = remove_stopwords(text)
    return ' '.join(words)


### Tokenization

In [8]:
accepted_answers_df = pd.read_csv('Accepted_Answers.csv')
accepted_answers_df

,Question,Relevant_Answer,question_id
0,InfoBar Component not visible after rendering,<p>You don't appear to be returning anything f...,73279519
1,Writing bits (not hex values) to file in Linux,"<p>perl to the rescue:</p>\n<pre class=""lang-b...",73414802
2,Change value of outerscope variable inside a f...,<p>Your variables are mutated from inside of a...,73370188
3,SmartContract/Solidity: The transaction has be...,<p><code>attributes[tokenId].lastClaimDate</co...,73364556
4,This name error keeps popping up despite me im...,<p>Conn is not defined here. Either change <co...,73417861
...,...,...,...
9995,"Spring Security Core authAjax, How do I ignore...",<p>When you make an un-authorized attempt to a...,5628199
9996,Dictionary+Queue Data Structure with Active Re...,<p>I'd start by just modeling the behavior you...,5888734
9997,Extend an existing list in Django settings,<p>You should be able to import form <code>dja...,5916209
9998,Python: How to deal with non-subscriptable obj...,<p>The way to fix it is to use dictionaries pr...,5977165


In [9]:
scored_answer_df = pd.read_csv('Scored_Answers.csv')
scored_answer_df

,Question,Relevant_Answer,question_id
0,in android how to send and receive special sms...,<p>For this you have to create your custom bro...,29165268
1,Python Salary Program Using Conditionals,<p>You're using full quotes where you should b...,29089187
2,How can I show a spinner in a ViewPager when t...,<p>I would do a check for this inside the <cod...,29169909
3,Android: Horizontal list view on custom adaptor,<blockquote>\n <p>layout whose one item cover...,28878917
4,"Even If Testcase failed It is going in ""onTest...",<p>Function <code>verify</code> won't make a t...,29300800
...,...,...,...
9995,Using Mean.io - How to disable JS files aggreg...,<p>I am using the version 0.6.16. You can set ...,25698549
9996,CoreGraphics prevents frame modifications on o...,<p>Your question doesn't really have anything ...,25839033
9997,finding the average of the largest numbers inp...,<p>Using a <code>break;</code> instruction is ...,25815186
9998,Show Greek Date in English Format,<p>You can try using <code>[$-F800]</code> for...,25694275


In [10]:
ground_truth_df = pd.concat([accepted_answers_df, scored_answer_df], axis=0, ignore_index=True)
ground_truth_df

,Question,Relevant_Answer,question_id
0,InfoBar Component not visible after rendering,<p>You don't appear to be returning anything f...,73279519
1,Writing bits (not hex values) to file in Linux,"<p>perl to the rescue:</p>\n<pre class=""lang-b...",73414802
2,Change value of outerscope variable inside a f...,<p>Your variables are mutated from inside of a...,73370188
3,SmartContract/Solidity: The transaction has be...,<p><code>attributes[tokenId].lastClaimDate</co...,73364556
4,This name error keeps popping up despite me im...,<p>Conn is not defined here. Either change <co...,73417861
...,...,...,...
19995,Using Mean.io - How to disable JS files aggreg...,<p>I am using the version 0.6.16. You can set ...,25698549
19996,CoreGraphics prevents frame modifications on o...,<p>Your question doesn't really have anything ...,25839033
19997,finding the average of the largest numbers inp...,<p>Using a <code>break;</code> instruction is ...,25815186
19998,Show Greek Date in English Format,<p>You can try using <code>[$-F800]</code> for...,25694275


In [11]:
all_answers_df = pd.read_csv('All_answers.csv')
all_answers_df

,question_id,question_title,answer_id,answer_body
0,930,How do I connect to a database and loop over a...,951,<p>@Goyuix -- that's excellent for something w...
1,930,How do I connect to a database and loop over a...,65774,<p>If you are intending on reading a large num...
2,930,How do I connect to a database and loop over a...,20049,<p>If you are querying a SQL Server database (...
3,930,How do I connect to a database and loop over a...,940,<p>Very roughly and from memory since I don't ...
4,930,How do I connect to a database and loop over a...,1022,<p>That's definitely a good way to do it. But...
...,...,...,...,...
33424,73829515,Big Query first non null value record type,73830169,<p>Consider below option</p>\n<pre><code>selec...
33425,73829809,Change Wordpress colors based on logo main color,73829900,"<p>First of all, you need colors from the imag..."
33426,73831805,Why are methods not valid constructors?,73832791,<p>As pointed out in the comments on the origi...
33427,73834279,No ability to publish newer build for external...,73834304,<p>I believe that there is possible to do more...


In [12]:
ground_truth_df['Clean_Relevant_Answer'] = ground_truth_df['Relevant_Answer'].apply(clean_text)
ground_truth_df

,Question,Relevant_Answer,question_id,Clean_Relevant_Answer
0,InfoBar Component not visible after rendering,<p>You don't appear to be returning anything f...,73279519,dont appear returning anything infobar try con...
1,Writing bits (not hex values) to file in Linux,"<p>perl to the rescue:</p>\n<pre class=""lang-b...",73414802,perl rescue perl e print packb argv0 011010000...
2,Change value of outerscope variable inside a f...,<p>Your variables are mutated from inside of a...,73370188,variables mutated inside asynchronous closure ...
3,SmartContract/Solidity: The transaction has be...,<p><code>attributes[tokenId].lastClaimDate</co...,73364556,attributestokenidlastclaimdate uint attributes...
4,This name error keeps popping up despite me im...,<p>Conn is not defined here. Either change <co...,73417861,conn defined either change connectiontodb conn...
...,...,...,...,...
19995,Using Mean.io - How to disable JS files aggreg...,<p>I am using the version 0.6.16. You can set ...,25698549,using version 0616 set aggregatefalse configen...
19996,CoreGraphics prevents frame modifications on o...,<p>Your question doesn't really have anything ...,25839033,question doesnt really anything core graphics ...
19997,finding the average of the largest numbers inp...,<p>Using a <code>break;</code> instruction is ...,25815186,using break instruction possibility often cons...
19998,Show Greek Date in English Format,<p>You can try using <code>[$-F800]</code> for...,25694275,try using f800 englishus locale prefix format ...


In [13]:
all_answers_df['clean_answer_body'] = all_answers_df['answer_body'].apply(clean_text)
all_answers_df.head()

,question_id,question_title,answer_id,answer_body,clean_answer_body
0,930,How do I connect to a database and loop over a...,951,<p>@Goyuix -- that's excellent for something w...,thats excellent something written memory teste...
1,930,How do I connect to a database and loop over a...,65774,<p>If you are intending on reading a large num...,intending reading large number columns records...
2,930,How do I connect to a database and loop over a...,20049,<p>If you are querying a SQL Server database (...,querying sql server database version 7 replace...
3,930,How do I connect to a database and loop over a...,940,<p>Very roughly and from memory since I don't ...,roughly memory since dont code laptop using ol...
4,930,How do I connect to a database and loop over a...,1022,<p>That's definitely a good way to do it. But...,thats definitely good way happen using databas...


In [14]:
questions = np.array(ground_truth_df['Question']).astype(str)

all_answers = all_answers_df['clean_answer_body'].values.astype(str)

In [15]:
questions_answers = np.append(questions, all_answers, axis=None)

In [16]:
questions_answers.shape

(53429,)

In [17]:
del questions
del all_answers

In [18]:
## Tokenizer
num_words = 2000
maxlen = 30

tokenizer = Tokenizer(num_words = num_words, split=' ')
tokenizer.fit_on_texts(questions_answers)

In [19]:
## Loaded model for evaluation
bilstm_encoder = load_model('/content/drive/Shareddrives/SEM 4/DATA 298B Code/Encoder_BiLSTM.keras')

In [20]:
bilstm_encoder.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Encoder-Input (InputLayer)  [(None, 30)]              0         
                                                                 
 Body-Word-Embedding (Embed  (None, 30, 150)           300000    
 ding)                                                           
                                                                 
 bidirectional (Bidirection  (None, 256)               285696    
 al)                                                             
                                                                 
Total params: 585696 (2.23 MB)
Trainable params: 585696 (2.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## NDCG Evaluation with FAISS Ranking

In [21]:
!pip install faiss-cpu  # for CPU version
# or
!pip install faiss-gpu  # for GPU version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.9 MB/s eta 0:00:00


In [22]:
import faiss
import numpy as np

In [23]:
def new_query_processing(text):
  text_preprocess = clean_text(text)
  tokenized_input = tokenizer.texts_to_sequences([text_preprocess])
  padded_seq = pad_sequences(tokenized_input, maxlen=maxlen, padding='post')
  return padded_seq

In [24]:
question_embeddings = np.load('/content/drive/Shareddrives/SEM 4/DATA 298B Code/BiLSTM_question_embeddings_all.npy')

In [25]:
question_embeddings.shape

(20000, 1, 256)

In [28]:
# Convert question embeddings to a numpy matrix
question_embeddings_matrix = np.vstack(question_embeddings)
question_embeddings_matrix.shape

# # Get the dimension of the embeddings
dimension = question_embeddings_matrix.shape[1]

# Create a FAISS index
index = faiss.IndexFlatL2(dimension)

# # Add the question embeddings to the index
index.add(question_embeddings_matrix)


In [29]:
question_embeddings_matrix.shape

(20000, 256)

In [30]:
def encode(text, tokenizer, model):
  padded_seq = new_query_processing(text)
  embedding = model.predict(padded_seq)[0]
  return np.expand_dims(embedding, axis=0)

In [31]:
def find_top_similar_question_indices(user_input, tokenizer, model, index, k=5):
    # Encode the user input
    user_input_embedding = encode(user_input, tokenizer, model)

    # Search the index for the top k similar embeddings
    _, indices = index.search(user_input_embedding, k)

    # Flatten the indices array
    indices = indices.flatten()

    return indices


In [32]:
def find_top_similar_answer_indices(user_input, tokenizer, model, index, k=5):
    # Encode the user input
    user_input_embedding = encode(user_input, tokenizer, model)

    # Search the index for the top k similar embeddings
    _, indices = index.search(user_input_embedding, k)

    # Flatten the indices array
    indices = indices.flatten()

    return indices

#### Groundtruth Ranking

In [33]:
# scored_answer_df = pd.read_csv('/content/drive/Shareddrives/SEM 3/DATA298A  /Code/Scored_Answers.csv')
# scored_answer_df.drop(columns='Unnamed: 0', inplace=True)

# accepted_answers_df = pd.read_csv('/content/drive/Shareddrives/SEM 3/DATA298A  /Code/Accepted_Answers.csv')
# accepted_answers_df.drop(columns='Unnamed: 0', inplace=True)

In [34]:
# ground_truth_df = pd.concat([accepted_answers_df, scored_answer_df], axis=0, ignore_index=True)
# ground_truth_df

In [35]:
# ground_truth_df.to_csv('/content/drive/Shareddrives/SEM 4/DATA 298B Code/Groundtruth_relevant_answers.csv', index=False)

ground_truth_df = pd.read_csv('/content/drive/Shareddrives/SEM 4/DATA 298B Code/Groundtruth_relevant_answers.csv')

In [36]:
# taking 2000 random questions

twothousand_q = ground_truth_df.sample(2000, random_state=11)
twothousand_q

,Question,Relevant_Answer,question_id
2440,cumsum in numpy ndarray (edited),<p>The solution of @Ben.T juste above worked p...,62200330
7382,Should I always provide default value in php f...,<p>MySQL does not support having default value...,24361295
13064,How to use/install Command 'host' Docker Alpin...,<p>Alpine has a website called pkgs.alpinelinu...,66851224
18810,Lua String patterns - shorter code,<p>You can at least chain and line-wrap it:</p...,15137004
359,Trouble removing package from Visual Studio 2010,"<p>Here's a <a href=""http://docs.nuget.org/doc...",11960360
...,...,...,...
9714,How to display a text in android after a butto...,"<p>OK, fjust write onClick method in OnClickLi...",9123887
13991,Terraform - How do I reference my Output insid...,<p>The module name is <code>subnetwork</code> ...,70510621
3128,Need help parsing TypeScript syntax,"<p>It is a lot to unpack, here it is line by l...",62851642
11191,GameObject store its own location in a tile ba...,<p>I'd advise you to store the location in the...,16743466


In [38]:
# call big query for scores and answer bodies

all_answersscore_df = pd.read_csv('All_answers_scores.csv')
all_answersscore_df

,question_id,question_title,answer_id,answer_body,score
0,13128,How can I combine several C/C++ libraries into...,27676078,"<p>On Linux, <a href=""https://en.wikipedia.org...",8
1,13128,How can I combine several C/C++ libraries into...,13136,<p>I'm not sure how to physically combine them...,0
2,13128,How can I combine several C/C++ libraries into...,22041,"<p>Maybe I'm misunderstanding, but don't you o...",0
3,13128,How can I combine several C/C++ libraries into...,13140,"<p>On Unix like systems, the ld and ar utiliti...",9
4,13128,How can I combine several C/C++ libraries into...,13206,<p>You could extract the object files from eac...,26
...,...,...,...,...,...
33208,73831805,Why are methods not valid constructors?,73832791,<p>As pointed out in the comments on the origi...,2
33209,73834279,No ability to publish newer build for external...,73834304,<p>I believe that there is possible to do more...,0
33210,73837100,How can I run multiple expressions inside of a...,73837248,<p>You're getting a syntax error due to the as...,-1
33211,73838452,Pad the missing data with NA for two matrices,73838548,<p>You can use <code>match</code> to find the ...,1


In [39]:
eval_q_list = twothousand_q['Question'].tolist()

In [40]:
from sklearn.metrics import ndcg_score

In [41]:
### creating function for eval

def ndcg_evaluation(question_list):

  counter = 1
  for user_question in question_list:
    print(counter)
    counter += 1
    ndcg_total = []

    ## getting the top 5 indices - questions
    top_question_indices = find_top_similar_question_indices(user_question, tokenizer, bilstm_encoder, index)

    #### Groundtruth ranks
    ## initiate score dataframe
    scores_df = pd.DataFrame({'score':[], 'answer_id':[], 'answer_body' : []})
    scores_df['answer_id'] = scores_df['answer_id'].astype(int)

    ## get scores
    for ind in top_question_indices:
      if len(scores_df) <= 5:
        q_id = ground_truth_df.loc[ind, 'question_id']
        filtered_df = all_answersscore_df[all_answersscore_df['question_id'] == q_id]
        # Select 'score','answr body' and 'answer_id' columns
        filtered_df = filtered_df[['score', 'answer_id','answer_body']]
        scores_df = pd.concat([scores_df,filtered_df], axis=0, ignore_index=True)
      else:
        break

    # Sort the scores DataFrame based on 'score' in descending order
    sorted_df = scores_df.sort_values(by='score', ascending=False)
    result_df = sorted_df.head(5)
    result_df.reset_index(drop=True, inplace=True)
    result_df = result_df.copy()
    result_df.loc[:, 'gt_relevance'] = list(range(len(result_df)-1, -1, -1))

    answer_ids = result_df['answer_id'].to_list()

    ## getting answer bodies
    answers_text_list = result_df['answer_body'].to_list()
    answers_embeddings = [encode(answer, tokenizer, bilstm_encoder) for answer in answers_text_list]

    ## FAISS
    # Convert answer embeddings to a numpy matrix
    answer_embeddings_matrix = np.vstack(answers_embeddings)
    # Get the dimension of the embeddings
    dimension = answer_embeddings_matrix.shape[1]
    # Create a FAISS index
    answer_index = faiss.IndexFlatL2(dimension)
    # Add the question embeddings to the index
    answer_index.add(answer_embeddings_matrix)

    ## getting top answer indices
    top_answer_indices = find_top_similar_answer_indices(user_question, tokenizer, bilstm_encoder, answer_index)

    result_df['top_indices'] = top_answer_indices

    fetched_values = []
    for i in top_answer_indices:
      test_rel_score = result_df['gt_relevance'].iloc[i]
      fetched_values.append(test_rel_score)
    result_df['test_relevance'] = np.array(fetched_values)


    ndcg = ndcg_score([result_df['gt_relevance'].to_list()], [result_df['test_relevance'].to_list()])

    ndcg_total.append(ndcg)

  ndcg_mean = np.mean(ndcg_total)

  return round(ndcg_mean, 4)


BiLSTM Encoder model - FAISS Evaluation

In [43]:
overall_ndgc_score = []

In [44]:
## 0 - 500
bilstm_ndcg_score1 = ndcg_evaluation(eval_q_list[:500])
overall_ndgc_score.append(bilstm_ndcg_score1)

1
1/1 [==============================] - 0s 58ms/step
2
1/1 [==============================] - 0s 49ms/step
3
1/1 [==============================] - 0s 44ms/step


<ipython-input-7-85e08033c571>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


1/1 [==============================] - 0s 39ms/step
4
1/1 [==============================] - 0s 36ms/step


<ipython-input-7-85e08033c571>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


1/1 [==============================] - 0s 42ms/step
5
1/1 [==============================] - 0s 38ms/step
6
1/1 [==============================] - 0s 40ms/step
7
1/1 [==============================] - 0s 41ms/step
8
1/1 [==============================] - 0s 26ms/step
9
1/1 [==============================] - 0s 24ms/step
10
1/1 [==============================] - 0s 26ms/step
11
1/1 [==============================] - 0s 25ms/step
12
1/1 [==============================] - 0s 29ms/step
13
1/1 [==============================] - 0s 26ms/step
14
1/1 [==============================] - 0s 25ms/step
15
1/1 [==============================] - 0s 25ms/step


...

1/1 [==============================] - 0s 27ms/step
478
1/1 [==============================] - 0s 27ms/step
479
1/1 [==============================] - 0s 30ms/step
480
1/1 [==============================] - 0s 26ms/step
481
1/1 [==============================] - 0s 26ms/step


<ipython-input-7-85e08033c571>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


1/1 [==============================] - 0s 28ms/step
482
1/1 [==============================] - 0s 40ms/step


<ipython-input-7-85e08033c571>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


1/1 [==============================] - 0s 40ms/step
483
1/1 [==============================] - 0s 47ms/step
484
1/1 [==============================] - 0s 38ms/step
485
1/1 [==============================] - 0s 43ms/step
486
1/1 [==============================] - 0s 31ms/step
487
1/1 [==============================] - 0s 26ms/step
488
1/1 [==============================] - 0s 25ms/step
489
1/1 [==============================] - 0s 30ms/step
490
1/1 [==============================] - 0s 29ms/step
491
1/1 [==============================] - 0s 28ms/step
492
1/1 [==============================] - 0s 25ms/step
493
1/1 [==============================] - 0s 25ms/step
494
1/1 [==============================] - 0s 25ms/step
495
1/1 [==============================] - 0s 29ms/step
496
1/1 [==============================] - 0s 33ms/step
497
1/1 [==============================] - 0s 25ms/step
498
1/1 [==============================] - 0s 24ms/step
499
1/1 [==============================] - 0s 31ms/s

In [45]:
## 500 - 1000
bilstm_ndcg_score2 = ndcg_evaluation(eval_q_list[500:1000])
overall_ndgc_score.append(bilstm_ndcg_score2)

1
1/1 [==============================] - 0s 27ms/step
2
1/1 [==============================] - 0s 28ms/step
3
1/1 [==============================] - 0s 24ms/step
4
1/1 [==============================] - 0s 26ms/step
5
1/1 [==============================] - 0s 39ms/step
6
1/1 [==============================] - 0s 41ms/step
7
1/1 [==============================] - 0s 53ms/step
8
1/1 [==============================] - 0s 39ms/step
9
1/1 [==============================] - 0s 45ms/step
10
1/1 [==============================] - 0s 26ms/step
11
1/1 [==============================] - 0s 27ms/step
12
1/1 [==============================] - 0s 25ms/step
13
1/1 [==============================] - 0s 26ms/step
14
1/1 [==============================] - 0s 28ms/step
15
1/1 [==============================] - 0s 28ms/step
16
1/1 [==============================] - 0s 29ms/step
17
1/1 [==============================] - 0s 26ms/step
18
1/1 [==============================] - 0s 27ms/step
19
1/1 [===========

<ipython-input-7-85e08033c571>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


1/1 [==============================] - 0s 28ms/step
21
1/1 [==============================] - 0s 29ms/step


<ipython-input-7-85e08033c571>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


1/1 [==============================] - 0s 32ms/step
22
1/1 [==============================] - 0s 32ms/step
23
1/1 [==============================] - 0s 37ms/step


...

1/1 [==============================] - 0s 38ms/step
452
1/1 [==============================] - 0s 36ms/step
453
1/1 [==============================] - 0s 36ms/step
454
1/1 [==============================] - 0s 27ms/step
455
1/1 [==============================] - 0s 33ms/step
456
1/1 [==============================] - 0s 26ms/step
457
1/1 [==============================] - 0s 26ms/step
458
1/1 [==============================] - 0s 25ms/step
459
1/1 [==============================] - 0s 30ms/step


<ipython-input-7-85e08033c571>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


1/1 [==============================] - 0s 25ms/step
460
1/1 [==============================] - 0s 33ms/step


<ipython-input-7-85e08033c571>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


1/1 [==============================] - 0s 29ms/step
461
1/1 [==============================] - 0s 31ms/step
462
1/1 [==============================] - 0s 24ms/step
463
1/1 [==============================] - 0s 31ms/step
464
1/1 [==============================] - 0s 28ms/step
465
1/1 [==============================] - 0s 29ms/step
466
1/1 [==============================] - 0s 26ms/step
467
1/1 [==============================] - 0s 24ms/step
468
1/1 [==============================] - 0s 25ms/step
469
1/1 [==============================] - 0s 31ms/step
470
1/1 [==============================] - 0s 24ms/step
471
1/1 [==============================] - 0s 27ms/step
472
1/1 [==============================] - 0s 24ms/step
473
1/1 [==============================] - 0s 27ms/step
474
1/1 [==============================] - 0s 41ms/step
475
1/1 [==============================] - 0s 38ms/step
476
1/1 [==============================] - 0s 38ms/step
477
1/1 [==============================] - 0s 37ms/s

In [46]:
## 1000 - 1500
bilstm_ndcg_score3 = ndcg_evaluation(eval_q_list[1000:1500])
overall_ndgc_score.append(bilstm_ndcg_score3)

1
1/1 [==============================] - 0s 47ms/step
2
1/1 [==============================] - 0s 45ms/step
3
1/1 [==============================] - 0s 27ms/step
4
1/1 [==============================] - 0s 33ms/step


<ipython-input-7-85e08033c571>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


1/1 [==============================] - 0s 31ms/step
5
1/1 [==============================] - 0s 29ms/step


<ipython-input-7-85e08033c571>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


1/1 [==============================] - 0s 28ms/step
6
1/1 [==============================] - 0s 28ms/step
7
1/1 [==============================] - 0s 24ms/step
8
1/1 [==============================] - 0s 24ms/step
9
1/1 [==============================] - 0s 27ms/step
10
1/1 [==============================] - 0s 42ms/step
11
1/1 [==============================] - 0s 44ms/step
12
1/1 [==============================] - 0s 36ms/step
13
1/1 [==============================] - 0s 40ms/step
14
1/1 [==============================] - 0s 56ms/step
15
1/1 [==============================] - 0s 41ms/step


...

1/1 [==============================] - 0s 27ms/step
474
1/1 [==============================] - 0s 32ms/step
475
1/1 [==============================] - 0s 26ms/step
476
1/1 [==============================] - 0s 24ms/step
477
1/1 [==============================] - 0s 28ms/step
478
1/1 [==============================] - 0s 24ms/step
479
1/1 [==============================] - 0s 27ms/step
480
1/1 [==============================] - 0s 24ms/step
481
1/1 [==============================] - 0s 25ms/step
482
1/1 [==============================] - 0s 24ms/step
483
1/1 [==============================] - 0s 33ms/step
484
1/1 [==============================] - 0s 25ms/step
485
1/1 [==============================] - 0s 29ms/step
486
1/1 [==============================] - 0s 28ms/step
487
1/1 [==============================] - 0s 24ms/step
488
1/1 [==============================] - 0s 26ms/step
489
1/1 [==============================] - 0s 26ms/step
490
1/1 [==============================] - 0s 48ms/s

<ipython-input-7-85e08033c571>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


1/1 [==============================] - 0s 32ms/step
498
1/1 [==============================] - 0s 30ms/step


<ipython-input-7-85e08033c571>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


1/1 [==============================] - 0s 29ms/step
499
1/1 [==============================] - 0s 26ms/step
500
1/1 [==============================] - 0s 26ms/step


In [47]:
## 1500 - 2000
bilstm_ndcg_score4 = ndcg_evaluation(eval_q_list[1500:2000])
overall_ndgc_score.append(bilstm_ndcg_score4)

1
1/1 [==============================] - 0s 30ms/step
2
1/1 [==============================] - 0s 26ms/step
3
1/1 [==============================] - 0s 25ms/step
4
1/1 [==============================] - 0s 24ms/step
5
1/1 [==============================] - 0s 25ms/step
6
1/1 [==============================] - 0s 26ms/step
7
1/1 [==============================] - 0s 34ms/step
8
1/1 [==============================] - 0s 25ms/step
9
1/1 [==============================] - 0s 24ms/step
10
1/1 [==============================] - 0s 26ms/step
11
1/1 [==============================] - 0s 24ms/step
12
1/1 [==============================] - 0s 28ms/step
13
1/1 [==============================] - 0s 28ms/step
14
1/1 [==============================] - 0s 27ms/step
15
1/1 [==============================] - 0s 25ms/step
16
1/1 [==============================] - 0s 29ms/step
17
1/1 [==============================] - 0s 39ms/step
18
1/1 [==============================] - 0s 47ms/step
19
1/1 [===========

<ipython-input-7-85e08033c571>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


1/1 [==============================] - 0s 29ms/step
64
1/1 [==============================] - 0s 31ms/step


<ipython-input-7-85e08033c571>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


1/1 [==============================] - 0s 29ms/step
65
1/1 [==============================] - 0s 30ms/step
66
1/1 [==============================] - 0s 51ms/step
67
1/1 [==============================] - 0s 43ms/step
68
1/1 [==============================] - 0s 40ms/step
69
1/1 [==============================] - 0s 43ms/step
70
1/1 [==============================] - 0s 50ms/step
71
1/1 [==============================] - 0s 28ms/step
72
1/1 [==============================] - 0s 32ms/step
73
1/1 [==============================] - 0s 28ms/step
74
1/1 [==============================] - 0s 27ms/step
75
1/1 [==============================] - 0s 26ms/step
76
1/1 [==============================] - 0s 26ms/step
77
1/1 [==============================] - 0s 27ms/step
78
1/1 [==============================] - 0s 25ms/step
79
1/1 [==============================] - 0s 24ms/step
80
1/1 [==============================] - 0s 28ms/step
81
1/1 [==============================] - 0s 27ms/step
82
1/1 [=====

...

1/1 [==============================] - 0s 30ms/step
455
1/1 [==============================] - 0s 26ms/step
456
1/1 [==============================] - 0s 27ms/step
457
1/1 [==============================] - 0s 26ms/step
458
1/1 [==============================] - 0s 25ms/step
459
1/1 [==============================] - 0s 25ms/step
460
1/1 [==============================] - 0s 27ms/step
461
1/1 [==============================] - 0s 27ms/step
462
1/1 [==============================] - 0s 37ms/step
463
1/1 [==============================] - 0s 39ms/step
464
1/1 [==============================] - 0s 37ms/step
465
1/1 [==============================] - 0s 42ms/step
466
1/1 [==============================] - 0s 42ms/step
467
1/1 [==============================] - 0s 26ms/step
468
1/1 [==============================] - 0s 26ms/step
469
1/1 [==============================] - 0s 32ms/step
470
1/1 [==============================] - 0s 31ms/step
471
1/1 [==============================] - 0s 26ms/s

<ipython-input-7-85e08033c571>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


1/1 [==============================] - 0s 26ms/step
474
1/1 [==============================] - 0s 35ms/step


<ipython-input-7-85e08033c571>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


1/1 [==============================] - 0s 24ms/step
475
1/1 [==============================] - 0s 24ms/step
476
1/1 [==============================] - 0s 29ms/step
477
1/1 [==============================] - 0s 25ms/step
478
1/1 [==============================] - 0s 24ms/step
479
1/1 [==============================] - 0s 25ms/step
480
1/1 [==============================] - 0s 25ms/step
481
1/1 [==============================] - 0s 30ms/step
482
1/1 [==============================] - 0s 27ms/step
483
1/1 [==============================] - 0s 28ms/step
484
1/1 [==============================] - 0s 25ms/step
485
1/1 [==============================] - 0s 25ms/step
486
1/1 [==============================] - 0s 41ms/step
487
1/1 [==============================] - 0s 45ms/step
488
1/1 [==============================] - 0s 40ms/step
489
1/1 [==============================] - 0s 37ms/step
490
1/1 [==============================] - 0s 38ms/step
491
1/1 [==============================] - 0s 24ms/s

In [48]:
overall_ndgc_score

[0.689, 0.9, 0.6831, 0.6746]

In [49]:
print(f'The NDGC score of Encoder BiLSTM - FAISS model (for 2000 test questions) is {np.mean(overall_ndgc_score):.4f}')

The NDGC score of Encoder BiLSTM - FAISS model (for 2000 test questions) is 0.7367
